# MNIST(TensorFlow版本)

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()

class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()

        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷積層神經元（卷積核）數目
            kernel_size=[5, 5],     # 接受區的大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数

        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 1]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

num_epochs = 1
batch_size = 50
learning_rate = 0.01

data_loader = MNISTLoader()
model = CNN()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#print(model.summary())
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.312980
batch 1: loss 12.441854
batch 2: loss 5.605434
batch 3: loss 2.307137
batch 4: loss 2.292812
batch 5: loss 2.307375
batch 6: loss 2.299499
batch 7: loss 2.317246
batch 8: loss 2.296153
batch 9: loss 2.298928
batch 10: loss 2.305989
batch 11: loss 2.309182
batch 12: loss 2.274323
batch 13: loss 2.327966
batch 14: loss 2.311240
batch 15: loss 2.302894
batch 16: loss 2.363893
batch 17: loss 2.344068
batch 18: loss 2.324596
batch 19: loss 2.280692
batch 20: loss 2.314594
batch 21: loss 2.302614
batch 22: loss 2.314267
batch 23: loss 2.312832
batch 24: loss 2.309758
batch 25: loss 2.297209
batch 26: loss 2.307161
batch 27: loss 2.312736
batch 28: loss 2.290920
batch 29: loss 2.308201
batch 30: loss 2.313276
batch 31: loss 2.304058
batch 32: loss 2.297906
batch 33: loss 2.298638
batch 34: loss 2.305308
batch 35: loss 2.296111
batch 36: loss 2.292198
batch 37: loss 2.285738
batch 38: loss 2.285405
batch 39: loss 2.300639
batch 40: loss 2.291445
batch 41: loss 2.317694
b

In [2]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.102800


# MNIST(Keras版本)

In [4]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
tf.compat.v1.reset_default_graph()


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))


train_images, test_images = train_images / 255.0, test_images / 255.0

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # params = 32*9+32
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # params = 320*9+64
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1480 - accuracy: 0.9545
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0490 - accuracy: 0.9851
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0353 - accuracy: 0.9887
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0262 - accuracy: 0.9918
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0220 - accuracy: 0.9927


In [5]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0293 - accuracy: 0.9909
